# Genome Power Laws

The following notebook demonstrates how there is a general feature of genomes across organisms that beyond a certain k-length, all kmers in the genome converge towards a linear relationship between the frequency of kmers (depth of coverage, X axis) and the number of kmers with that frequency (# of kmers, Y axis). This is simply a transformed histogram, however it is an important feature to recognize because it can be leveraged to efficiently assemble genomes and identify the closest related species to the assemblies.

get taxonomy from [here](ftp://ftp.ncbi.nih.gov/pub/taxonomy/)

In [1]:
if !isdir("../_data/taxonomy")
    mkdir("../_data/taxonomy")
end
if !isfile("../_data/taxonomy/taxdump.tar.gz")
    download("ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz", "../_data/taxonomy/taxdump.tar.gz")
end

In [2]:
run(`tar -xzf ../_data/taxonomy/taxdump.tar.gz -C ../_data/taxonomy`)

Process(`tar -xzf ../_data/taxonomy/taxdump.tar.gz -C ../_data/taxonomy`, ProcessExited(0))

In [3]:
using uCSV
using DataFrames
using StatsBase
using Random

In [4]:
refseq_metadata = DataFrame(uCSV.read("../_data/assembly_summary_refseq.txt", delim="\t", header=2))
reference_metadata = 
    refseq_metadata[
        (refseq_metadata[:version_status] .== "latest") .&
        (refseq_metadata[:genome_rep] .== "Full") .&
        (refseq_metadata[:refseq_category] .== "reference genome")
        , :]
first(reference_metadata, 6)

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material
,String,String,String,String,String,Int64,Int64,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String
1,GCF_000001215.4,PRJNA164,SAMN02803731,,reference genome,7227,7227,Drosophila melanogaster,,,latest,Chromosome,Major,Full,2014/08/01,Release 6 plus ISO1 MT,The FlyBase Consortium/Berkeley Drosophila Genome Project/Celera Genomics,GCA_000001215.4,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/215/GCF_000001215.4_Release_6_plus_ISO1_MT,,
2,GCF_000001405.39,PRJNA168,,,reference genome,9606,9606,Homo sapiens,,,latest,Chromosome,Patch,Full,2019/02/28,GRCh38.p13,Genome Reference Consortium,GCA_000001405.28,different,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.39_GRCh38.p13,,
3,GCF_000001635.26,PRJNA169,,,reference genome,10090,10090,Mus musculus,,,latest,Chromosome,Patch,Full,2017/09/15,GRCm38.p6,Genome Reference Consortium,GCA_000001635.8,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.26_GRCm38.p6,,
4,GCF_000001735.4,PRJNA116,SAMN03081427,,reference genome,3702,3702,Arabidopsis thaliana,ecotype=Columbia,,latest,Chromosome,Minor,Full,2018/03/15,TAIR10.1,The Arabidopsis Information Resource (TAIR),GCA_000001735.2,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/735/GCF_000001735.4_TAIR10.1,,
5,GCF_000002035.6,PRJNA13922,SAMN06930106,,reference genome,7955,7955,Danio rerio,,,latest,Chromosome,Major,Full,2017/05/09,GRCz11,Genome Reference Consortium,GCA_000002035.4,different,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/002/035/GCF_000002035.6_GRCz11,,
6,GCF_000002985.6,PRJNA158,SAMEA3138177,,reference genome,6239,6239,Caenorhabditis elegans,strain=Bristol N2,,latest,Complete Genome,Major,Full,2013/02/07,WBcel235,C. elegans Sequencing Consortium,GCA_000002985.3,different,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/002/985/GCF_000002985.6_WBcel235,,


get node id, rank, and division ID from nodes.dmp

In [5]:
columns = first(uCSV.read("../_data/taxonomy/nodes.dmp", delim="|", trimwhitespace=true))[[1, 3, 5]]
column_names = [:tax_id, :rank, :division_id]
nodes = DataFrame(columns, column_names)
first(nodes, 6)

,tax_id,rank,division_id
,Int64,String,Int64
1,1,no rank,8
2,2,superkingdom,0
3,6,genus,0
4,7,species,0
5,9,species,0
6,10,genus,0


get scientific name from names.dmp

- 1 => tax_id
- 2 => name
- 4 => name_class

drop extra info columns and only keep columns mapping species name and ID

In [6]:
columns = first(uCSV.read("../_data/taxonomy/names.dmp", delim='|', trimwhitespace=true))[[1, 2, 4]]
column_names = [:tax_id, :name, :name_class]
names = DataFrame(columns, column_names)
names = names[names[:name_class] .== "scientific name", :]
first(names, 6)

,tax_id,name,name_class
,Int64,String,String
1,1,root,scientific name
2,2,Bacteria,scientific name
3,6,Azorhizobium,scientific name
4,7,Azorhizobium caulinodans,scientific name
5,9,Buchnera aphidicola,scientific name
6,10,Cellvibrio,scientific name


get division name from division.dmp

- division_id => 1
- division_code => 2
- division_name => 3

In [7]:
columns = first(uCSV.read("../_data/taxonomy/division.dmp", delim='|', trimwhitespace=true))[[1, 2, 3]]
column_names = [:division_id, :division_code, :division_name]
divisions = DataFrame(columns, column_names)
first(divisions, 6)

,division_id,division_code,division_name
,Int64,String,String
1,0,BCT,Bacteria
2,1,INV,Invertebrates
3,2,MAM,Mammals
4,3,PHG,Phages
5,4,PLN,Plants and Fungi
6,5,PRI,Primates


In [8]:
taxonomy_info = join(join(nodes, names; on = :tax_id, kind = :inner), divisions, on = :division_id, kind = :inner)
first(taxonomy_info, 6)

,tax_id,rank,division_id,name,name_class,division_code,division_name
,Int64,String,Int64,String,String,String,String
1,1,no rank,8,root,scientific name,UNA,Unassigned
2,2,superkingdom,0,Bacteria,scientific name,BCT,Bacteria
3,6,genus,0,Azorhizobium,scientific name,BCT,Bacteria
4,7,species,0,Azorhizobium caulinodans,scientific name,BCT,Bacteria
5,9,species,0,Buchnera aphidicola,scientific name,BCT,Bacteria
6,10,genus,0,Cellvibrio,scientific name,BCT,Bacteria


In [9]:
merged = join(taxonomy_info, reference_metadata, on = (:tax_id => :species_taxid), kind=:inner)
first(merged, 6)

,tax_id,rank,division_id,name,name_class,division_code,division_name,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material
,Int64,String,Int64,String,String,String,String,String,String,String,String,String,Int64,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String
1,9,species,0,Buchnera aphidicola,scientific name,BCT,Bacteria,GCF_000009605.1,PRJNA57805,SAMD00061095,,reference genome,107806,Buchnera aphidicola str. APS (Acyrthosiphon pisum),strain=APS,Tokyo1998,latest,Complete Genome,Major,Full,2004/05/11,ASM960v1,Rikken GSC,GCA_000009605.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/009/605/GCF_000009605.1_ASM960v1,,
2,139,species,0,Borreliella burgdorferi,scientific name,BCT,Bacteria,GCF_000008685.2,PRJNA57581,SAMN02603966,,reference genome,224326,Borreliella burgdorferi B31,strain=B31,,latest,Complete Genome,Major,Full,2011/11/09,ASM868v2,TIGR,GCA_000008685.2,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/008/685/GCF_000008685.2_ASM868v2,,assembly from type material
3,158,species,0,Treponema denticola,scientific name,BCT,Bacteria,GCF_000008185.1,PRJNA57583,SAMN02603967,,reference genome,243275,Treponema denticola ATCC 35405,strain=ATCC 35405,,latest,Complete Genome,Major,Full,2004/02/03,ASM818v1,TIGR,GCA_000008185.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/008/185/GCF_000008185.1_ASM818v1,,assembly from type material
4,173,species,0,Leptospira interrogans,scientific name,BCT,Bacteria,GCF_000092565.1,PRJNA57881,SAMN02603127,,reference genome,189518,Leptospira interrogans serovar Lai str. 56601,strain=56601,,latest,Complete Genome,Major,Full,2010/04/06,ASM9256v1,"Chinese National HGC, Shanghai",GCA_000092565.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/092/565/GCF_000092565.1_ASM9256v1,,
5,197,species,0,Campylobacter jejuni,scientific name,BCT,Bacteria,GCF_000009085.1,PRJNA57587,SAMEA1705929,,reference genome,192222,Campylobacter jejuni subsp. jejuni NCTC 11168 = ATCC 700819,strain=NCTC 11168,,latest,Complete Genome,Major,Full,2003/05/06,ASM908v1,Sanger Institute,GCA_000009085.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/009/085/GCF_000009085.1_ASM908v1,,
6,210,species,0,Helicobacter pylori,scientific name,BCT,Bacteria,GCF_000008525.1,PRJNA57787,SAMN02603995,,reference genome,85962,Helicobacter pylori 26695,strain=26695,,latest,Complete Genome,Major,Full,1999/12/22,ASM852v1,TIGR,GCA_000008525.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/008/525/GCF_000008525.1_ASM852v1,,


How many in each group?

In [10]:
countmap(merged[:division_name])

Dict{String,Int64} with 8 entries:
  "Bacteria"         => 120
  "Rodents"          => 1
  "Vertebrates"      => 1
  "Primates"         => 1
  "Invertebrates"    => 2
  "Phages"           => 5
  "Plants and Fungi" => 2
  "Viruses"          => 41

Randomly select one organism from each group of organisms

In [11]:
Random.seed!(3)
for division in unique(merged[:division_name])
    if division == "Plants and Fungi" || division == "Invertebrates"
        subset = merged[findall(merged[:division_name] .== division), :]
        for row in eachrow(subset)
            println(join([division, row[:name], basename(row[:ftp_path]) * "_genomic.fna.gz"], "\t\t"))
        end
    else
        subset = merged[rand(findall(merged[:division_name] .== division)), :]
        println(join([division, subset[:name], basename(subset[:ftp_path]) * "_genomic.fna.gz"], " \t\t"))
    end
end

Bacteria 		Flavobacterium psychrophilum 		GCF_000064305.2_ASM6430v2_genomic.fna.gz
Plants and Fungi		Arabidopsis thaliana		GCF_000001735.4_TAIR10.1_genomic.fna.gz
Plants and Fungi		Saccharomyces cerevisiae		GCF_000146045.2_R64_genomic.fna.gz
Invertebrates		Caenorhabditis elegans		GCF_000002985.6_WBcel235_genomic.fna.gz
Invertebrates		Drosophila melanogaster		GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.gz
Vertebrates 		Danio rerio 		GCF_000002035.6_GRCz11_genomic.fna.gz
Primates 		Homo sapiens 		GCF_000001405.39_GRCh38.p13_genomic.fna.gz
Rodents 		Mus musculus 		GCF_000001635.26_GRCm38.p6_genomic.fna.gz
Phages 		Chlamydia virus Chp2 		GCF_000849665.1_ViralProj14593_genomic.fna.gz
Viruses 		Norwalk virus 		GCF_000868425.1_ViralProj17577_genomic.fna.gz


```bash
# Phages 		Chlamydia virus Chp2 		
FASTA=GCF_000849665.1_ViralProj14593_genomic.fna
# all kmers >= k=17 are unique
K_RANGE="7 11 13 17"
parallel Eisenia\ stream-kmers\ --k\ \{1\}\ --fasta\ $FASTA.gz\ \|\ sort\ --temporary-directory\ \.\ --compress-program\ gzip \|\ uniq\ --count\ \| gzip\ \>\ $FASTA.K\{1\}.counts.gz ::: $K_RANGE
parallel gzip\ --decompress\ --stdout\ $FASTA.K\{1\}.counts.gz\ \|\ awk\ \'\{print\ \$1\}\'\ \|\ sort\ --numeric\ \|\ uniq\ --count\ \>\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
parallel Eisenia\ plot\ histogram\ --histogram\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
```

![](GCF_000849665.1_ViralProj14593_genomic.fna.K7.counts.histogram.svg)
![](GCF_000849665.1_ViralProj14593_genomic.fna.K11.counts.histogram.svg)
![](GCF_000849665.1_ViralProj14593_genomic.fna.K13.counts.histogram.svg)
![](GCF_000849665.1_ViralProj14593_genomic.fna.K17.counts.histogram.svg)

```bash
# Viruses 		Norwalk virus
FASTA=GCF_000868425.1_ViralProj17577_genomic.fna
# all kmers >= k=17 are unique
K_RANGE="7 11 13 17"
parallel Eisenia\ stream-kmers\ --k\ \{1\}\ --fasta\ $FASTA.gz\ \|\ sort\ --temporary-directory\ \.\ --compress-program\ gzip \|\ uniq\ --count\ \| gzip\ \>\ $FASTA.K\{1\}.counts.gz ::: $K_RANGE
parallel gzip\ --decompress\ --stdout\ $FASTA.K\{1\}.counts.gz\ \|\ awk\ \'\{print\ \$1\}\'\ \|\ sort\ --numeric\ \|\ uniq\ --count\ \>\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
parallel Eisenia\ plot\ histogram\ --histogram\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
```

![](GCF_000868425.1_ViralProj17577_genomic.fna.K7.counts.histogram.svg)
![](GCF_000868425.1_ViralProj17577_genomic.fna.K11.counts.histogram.svg)
![](GCF_000868425.1_ViralProj17577_genomic.fna.K13.counts.histogram.svg)
![](GCF_000868425.1_ViralProj17577_genomic.fna.K17.counts.histogram.svg)

```bash
# Bacteria 		Flavobacterium psychrophilum
FASTA=GCF_000064305.2_ASM6430v2_genomic.fna
K_RANGE="7 11 13 17 19 23 29 31"
parallel Eisenia\ stream-kmers\ --k\ \{1\}\ --fasta\ $FASTA.gz\ \|\ sort\ --temporary-directory\ \.\ --compress-program\ gzip \|\ uniq\ --count\ \| gzip\ \>\ $FASTA.K\{1\}.counts.gz ::: $K_RANGE
parallel gzip\ --decompress\ --stdout\ $FASTA.K\{1\}.counts.gz\ \|\ awk\ \'\{print\ \$1\}\'\ \|\ sort\ --numeric\ \|\ uniq\ --count\ \>\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
parallel Eisenia\ plot\ histogram\ --histogram\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
```

![](GCF_000064305.2_ASM6430v2_genomic.fna.K7.counts.histogram.svg)
![](GCF_000064305.2_ASM6430v2_genomic.fna.K11.counts.histogram.svg)
![](GCF_000064305.2_ASM6430v2_genomic.fna.K13.counts.histogram.svg)
![](GCF_000064305.2_ASM6430v2_genomic.fna.K17.counts.histogram.svg)
![](GCF_000064305.2_ASM6430v2_genomic.fna.K19.counts.histogram.svg)
![](GCF_000064305.2_ASM6430v2_genomic.fna.K23.counts.histogram.svg)
![](GCF_000064305.2_ASM6430v2_genomic.fna.K29.counts.histogram.svg)
![](GCF_000064305.2_ASM6430v2_genomic.fna.K31.counts.histogram.svg)

```bash
# Plants and Fungi		Saccharomyces cerevisiae
FASTA=GCF_000146045.2_R64_genomic.fna
K_RANGE="7 11 13 17 19 23 29 31"
parallel Eisenia\ stream-kmers\ --k\ \{1\}\ --fasta\ $FASTA.gz\ \|\ sort\ --temporary-directory\ \.\ --compress-program\ gzip \|\ uniq\ --count\ \| gzip\ \>\ $FASTA.K\{1\}.counts.gz ::: $K_RANGE
parallel gzip\ --decompress\ --stdout\ $FASTA.K\{1\}.counts.gz\ \|\ awk\ \'\{print\ \$1\}\'\ \|\ sort\ --numeric\ \|\ uniq\ --count\ \>\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
parallel Eisenia\ plot\ histogram\ --histogram\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
```

![](GCF_000146045.2_R64_genomic.fna.K7.counts.histogram.svg)
![](GCF_000146045.2_R64_genomic.fna.K11.counts.histogram.svg)
![](GCF_000146045.2_R64_genomic.fna.K13.counts.histogram.svg)
![](GCF_000146045.2_R64_genomic.fna.K17.counts.histogram.svg)
![](GCF_000146045.2_R64_genomic.fna.K19.counts.histogram.svg)
![](GCF_000146045.2_R64_genomic.fna.K23.counts.histogram.svg)
![](GCF_000146045.2_R64_genomic.fna.K29.counts.histogram.svg)
![](GCF_000146045.2_R64_genomic.fna.K31.counts.histogram.svg)

```bash
# Invertebrates		Caenorhabditis elegans
FASTA=GCF_000002985.6_WBcel235_genomic.fna
K_RANGE="7 11 13 17 19 23 29 31"
parallel Eisenia\ stream-kmers\ --k\ \{1\}\ --fasta\ $FASTA.gz\ \|\ sort\ --temporary-directory\ \.\ --compress-program\ gzip \|\ uniq\ --count\ \| gzip\ \>\ $FASTA.K\{1\}.counts.gz ::: $K_RANGE
parallel gzip\ --decompress\ --stdout\ $FASTA.K\{1\}.counts.gz\ \|\ awk\ \'\{print\ \$1\}\'\ \|\ sort\ --numeric\ \|\ uniq\ --count\ \>\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
parallel Eisenia\ plot\ histogram\ --histogram\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
```

![](GCF_000002985.6_WBcel235_genomic.fna.K7.counts.histogram.svg)
![](GCF_000002985.6_WBcel235_genomic.fna.K11.counts.histogram.svg)
![](GCF_000002985.6_WBcel235_genomic.fna.K13.counts.histogram.svg)
![](GCF_000002985.6_WBcel235_genomic.fna.K17.counts.histogram.svg)
![](GCF_000002985.6_WBcel235_genomic.fna.K19.counts.histogram.svg)
![](GCF_000002985.6_WBcel235_genomic.fna.K23.counts.histogram.svg)
![](GCF_000002985.6_WBcel235_genomic.fna.K29.counts.histogram.svg)
![](GCF_000002985.6_WBcel235_genomic.fna.K31.counts.histogram.svg)

```bash
# Plants and Fungi		Arabidopsis thaliana
FASTA=GCF_000001735.4_TAIR10.1_genomic.fna
K_RANGE="7 11 13 17 19 23 29 31"
parallel Eisenia\ stream-kmers\ --k\ \{1\}\ --fasta\ $FASTA.gz\ \|\ sort\ --temporary-directory\ \.\ --compress-program\ gzip \|\ uniq\ --count\ \| gzip\ \>\ $FASTA.K\{1\}.counts.gz ::: $K_RANGE
parallel gzip\ --decompress\ --stdout\ $FASTA.K\{1\}.counts.gz\ \|\ awk\ \'\{print\ \$1\}\'\ \|\ sort\ --numeric\ \|\ uniq\ --count\ \>\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
parallel Eisenia\ plot\ histogram\ --histogram\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
```

![](GCF_000001735.4_TAIR10.1_genomic.fna.K7.counts.histogram.svg)

Something really interesting is happening here. Notice the two distributions that are present in the K=7 histogram graph? The first distribution is loglinear and the second distribution is lognormal. All previous gnomes in this group only have the second lognormal distribution. This is worth further investigation.

![](GCF_000001735.4_TAIR10.1_genomic.fna.K11.counts.histogram.svg)
![](GCF_000001735.4_TAIR10.1_genomic.fna.K13.counts.histogram.svg)
![](GCF_000001735.4_TAIR10.1_genomic.fna.K17.counts.histogram.svg)
![](GCF_000001735.4_TAIR10.1_genomic.fna.K19.counts.histogram.svg)
![](GCF_000001735.4_TAIR10.1_genomic.fna.K23.counts.histogram.svg)
![](GCF_000001735.4_TAIR10.1_genomic.fna.K29.counts.histogram.svg)
![](GCF_000001735.4_TAIR10.1_genomic.fna.K31.counts.histogram.svg)

```bash
# Invertebrates		Drosophila melanogaster
FASTA=GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna
K_RANGE="7 11 13 17 19 23 29 31"
parallel Eisenia\ stream-kmers\ --k\ \{1\}\ --fasta\ $FASTA.gz\ \|\ sort\ --temporary-directory\ \.\ --compress-program\ gzip \|\ uniq\ --count\ \| gzip\ \>\ $FASTA.K\{1\}.counts.gz ::: $K_RANGE
parallel gzip\ --decompress\ --stdout\ $FASTA.K\{1\}.counts.gz\ \|\ awk\ \'\{print\ \$1\}\'\ \|\ sort\ --numeric\ \|\ uniq\ --count\ \>\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
parallel Eisenia\ plot\ histogram\ --histogram\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
```

![](GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.K7.counts.histogram.svg)
![](GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.K11.counts.histogram.svg)
![](GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.K13.counts.histogram.svg)
![](GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.K17.counts.histogram.svg)
![](GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.K19.counts.histogram.svg)
![](GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.K23.counts.histogram.svg)
![](GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.K29.counts.histogram.svg)
![](GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.K31.counts.histogram.svg)

```bash
# Vertebrates 		Danio rerio
FASTA=GCF_000002035.6_GRCz11_genomic.fna
K_RANGE="7 11 13 17 19 23 29 31"
parallel Eisenia\ stream-kmers\ --k\ \{1\}\ --fasta\ $FASTA.gz\ \|\ sort\ --temporary-directory\ \.\ --compress-program\ gzip \|\ uniq\ --count\ \| gzip\ \>\ $FASTA.K\{1\}.counts.gz ::: $K_RANGE
parallel gzip\ --decompress\ --stdout\ $FASTA.K\{1\}.counts.gz\ \|\ awk\ \'\{print\ \$1\}\'\ \|\ sort\ --numeric\ \|\ uniq\ --count\ \>\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
parallel Eisenia\ plot\ histogram\ --histogram\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
```

Still calculating...
<!--
![](../_data/refseq_reference_genomes/GCF_000002035.6_GRCz11_genomic.fna.K7.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000002035.6_GRCz11_genomic.fna.K11.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000002035.6_GRCz11_genomic.fna.K13.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000002035.6_GRCz11_genomic.fna.K17.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000002035.6_GRCz11_genomic.fna.K19.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000002035.6_GRCz11_genomic.fna.K23.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000002035.6_GRCz11_genomic.fna.K29.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000002035.6_GRCz11_genomic.fna.K31.counts.histogram.svg)
-->

```bash
# Rodents 		Mus musculus
FASTA=GCF_000001635.26_GRCm38.p6_genomic.fna
K_RANGE="7 11 13 17 19 23 29 31"
parallel Eisenia\ stream-kmers\ --k\ \{1\}\ --fasta\ $FASTA.gz\ \|\ sort\ --temporary-directory\ \.\ --compress-program\ gzip \|\ uniq\ --count\ \| gzip\ \>\ $FASTA.K\{1\}.counts.gz ::: $K_RANGE
parallel gzip\ --decompress\ --stdout\ $FASTA.K\{1\}.counts.gz\ \|\ awk\ \'\{print\ \$1\}\'\ \|\ sort\ --numeric\ \|\ uniq\ --count\ \>\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
parallel Eisenia\ plot\ histogram\ --histogram\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
```

Still calculating...
<!--
![](../_data/refseq_reference_genomes/GCF_000001635.26_GRCm38.p6_genomic.fna.K7.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000001635.26_GRCm38.p6_genomic.fna.K11.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000001635.26_GRCm38.p6_genomic.fna.K13.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000001635.26_GRCm38.p6_genomic.fna.K17.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000001635.26_GRCm38.p6_genomic.fna.K19.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000001635.26_GRCm38.p6_genomic.fna.K23.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000001635.26_GRCm38.p6_genomic.fna.K29.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000001635.26_GRCm38.p6_genomic.fna.K31.counts.histogram.svg)
-->

```bash
# Primates 		Homo sapiens
FASTA=GCF_000001405.39_GRCh38.p13_genomic.fna
K_RANGE="7 11 13 17 19 23 29 31"
parallel Eisenia\ stream-kmers\ --k\ \{1\}\ --fasta\ $FASTA.gz\ \|\ sort\ --temporary-directory\ \.\ --compress-program\ gzip \|\ uniq\ --count\ \| gzip\ \>\ $FASTA.K\{1\}.counts.gz ::: $K_RANGE
parallel gzip\ --decompress\ --stdout\ $FASTA.K\{1\}.counts.gz\ \|\ awk\ \'\{print\ \$1\}\'\ \|\ sort\ --numeric\ \|\ uniq\ --count\ \>\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
parallel Eisenia\ plot\ histogram\ --histogram\ $FASTA.K\{1\}.counts.histogram ::: $K_RANGE
```

Still calculating...
<!--
![](../_data/refseq_reference_genomes/GCF_000001405.39_GRCh38.p13_genomic.fna.K7.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000001405.39_GRCh38.p13_genomic.fna.K11.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000001405.39_GRCh38.p13_genomic.fna.K13.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000001405.39_GRCh38.p13_genomic.fna.K17.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000001405.39_GRCh38.p13_genomic.fna.K19.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000001405.39_GRCh38.p13_genomic.fna.K23.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000001405.39_GRCh38.p13_genomic.fna.K29.counts.histogram.svg)
![](../_data/refseq_reference_genomes/GCF_000001405.39_GRCh38.p13_genomic.fna.K31.counts.histogram.svg)
-->

Additional work forthcoming includes:
- programmatically finding the smallest k-length that obtains a linear log-log kmer histogram relationship
- simulating new genomes that utilize this kmer distribution
- determining whether this minimal linear log-log k value is the optimal starting value for genome assembly and identification in all cases, or just some cases
- assessing how different error profiles from different sequencing technologies affect this count-frequency relationship (e.g. converging to zipfs law?)
- assessing how this count-frequency relationship changes in mixed metagenomic communities